This Code Handles From Detection to Creating a Database.

# Import dependencies

In [1]:
from ultralytics import YOLO 

In [2]:
import cv2
import os
import easyocr
import re
import datetime
import csv

# EasyOCR application

In [6]:
def extract_text_from_image(image_path):
    # Initialize the EasyOCR reader
    reader = easyocr.Reader(['en'])

    # Read the image and extract text
    result = reader.readtext(image_path)

    # Extract all text
    extracted_text = []
    for detection in result:
        text = detection[1]
        extracted_text.append(text)

    return extracted_text

# Car type class dictionary

In [7]:
class_names = {
    0: 'Hatchback',
    1: 'MidSize-SUV',
    3: 'SUV',
    4: 'Sedan'
}

# Load the model

In [8]:
model = YOLO('./best.pt')

# car detection and csv updation

# Parking modes: Parking or Leaving

The code compares the histogram of the current frame with the histogram of the previous frame 

This comparison is done to measure the similarity between the histograms. A correlation value is obtained, which indicates the degree of resemblance between the two histograms.

If the correlation exceeds a certain threshold, the current frame is considered similar to the previous frame, and further processing is skipped. 

This helps to reduce unnecessary computation and focus on frames with significant changes.

By using histogram correlation, the code decides whether to skip the object detection process for a frame.

If the correlation value is higher than the threshold, it implies that the frame is similar to the previous one.

Skipping the detection process for such frames improves efficiency.


In [ ]:

mode = "Parking"
threshold = 0.9  # Histogram correlation threshold
video_path = "entry2.mp4"

# Open the video file
cap = cv2.VideoCapture(video_path)

# Initialize variables for the previous frame and its histogram
previous_frame = None
previous_histogram = None

# List to store the results for each frame
results_list = []
added_number_plates = []
added_car_types = []

# Create a CSV file with Car Type, Number, and Car Status
with open('car_data.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Car Type', 'Number', 'Car Status'])

    # Read frames from the video
    while cap.isOpened():
        ret, frame = cap.read()

        if not ret:
            break

        # Convert the frame to grayscale
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Calculate the histogram of the current frame
        current_histogram = cv2.calcHist([gray_frame], [0], None, [256], [0, 256])

        # Normalize the histogram (optional but recommended)
        cv2.normalize(current_histogram, current_histogram, 0, 1, cv2.NORM_MINMAX)

        # Compare histogram correlation with the previous frame
        if previous_histogram is not None:
            correlation = cv2.compareHist(previous_histogram, current_histogram, cv2.HISTCMP_CORREL)
            if correlation > threshold:
                # Skip detection for this frame
                continue

        # Store the current frame and histogram for the next iteration
        previous_frame = gray_frame
        previous_histogram = current_histogram

        # Run YOLOv8 inference on the frame
        results = model(frame)
        cls = []
        number_plate = ""
        detected_car_type = ""

        for result in results:
            boxes = result.boxes.cpu().numpy()
            for i, box in enumerate(boxes):
                if box.conf[0] > 0.4:
                    cls = int(box.cls[0])
                    if cls == 2:
                        r = box.xyxy[0].astype(int)
                        crop = frame[r[1]:r[3], r[0]:r[2]]
                        cv2.imwrite("numberplate.jpg", crop)
                        # Specify the path to the image
                        number_plate = 'numberplate.jpg'
                    elif cls != 2:
                        car_type = class_names.get(cls)
                        print(f"Car Type: {car_type}")
                        detected_car_type = car_type

        # Extract text from the image (implement your extract_text_from_image function)
        text = extract_text_from_image(number_plate)
        text = str(text)

        # Define the pattern for number plate extraction using regular expressions
        pattern = r'.*'

        # Search for the pattern in the extracted text
        extracted_number_plates = re.findall(pattern, text)

        # Iterate over the extracted number plates
        for plate in extracted_number_plates:
            extracted_number_plate = str(plate)  # Convert the number plate to a string

            # Check if the number plate is already added
            if extracted_number_plate in added_number_plates:
                continue  # Skip the current iteration if the number plate is already added

            if detected_car_type in added_car_types:
                continue

            # Append the number plate to the added_number_plates list
            added_number_plates.append(extracted_number_plate)

            # Append the car type to the added car type list
            added_car_types.append(detected_car_type)

            # Append the results to the results list
            results_list.append((detected_car_type, extracted_number_plate))

            if mode == "Parking":
                 writer.writerow([detected_car_type, extracted_number_plate.strip("[]"), "1"])  # Convert "1" to string and remove square brackets
            else:
                writer.writerow([detected_car_type, extracted_number_plate.strip("[]"), "0"])  # Convert "0" to string and remove square brackets

# Release the video capture
cap.release()

In [12]:

mode = "Leaving"
threshold = 0.9  # Histogram correlation threshold
video_path = "entry2.mp4"

# Open the video file
cap = cv2.VideoCapture(video_path)

# Initialize variables for the previous frame and its histogram
previous_frame = None
previous_histogram = None

# List to store the results for each frame
results_list = []
added_number_plates = []
added_car_types = []

# Create a CSV file with Car Type, Number, and Car Status
with open('car_data.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Car Type', 'Number', 'Car Status'])

    # Read frames from the video
    while cap.isOpened():
        ret, frame = cap.read()

        if not ret:
            break

        # Convert the frame to grayscale
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Calculate the histogram of the current frame
        current_histogram = cv2.calcHist([gray_frame], [0], None, [256], [0, 256])

        # Normalize the histogram (optional but recommended)
        cv2.normalize(current_histogram, current_histogram, 0, 1, cv2.NORM_MINMAX)

        # Compare histogram correlation with the previous frame
        if previous_histogram is not None:
            correlation = cv2.compareHist(previous_histogram, current_histogram, cv2.HISTCMP_CORREL)
            if correlation > threshold:
                # Skip detection for this frame
                continue

        # Store the current frame and histogram for the next iteration
        previous_frame = gray_frame
        previous_histogram = current_histogram

        # Run YOLOv8 inference on the frame
        results = model(frame)
        cls = []
        number_plate = ""
        detected_car_type = ""

        for result in results:
            boxes = result.boxes.cpu().numpy()
            for i, box in enumerate(boxes):
                if box.conf[0] > 0.4:
                    cls = int(box.cls[0])
                    if cls == 2:
                        r = box.xyxy[0].astype(int)
                        crop = frame[r[1]:r[3], r[0]:r[2]]
                        cv2.imwrite("numberplate.jpg", crop)
                        # Specify the path to the image
                        number_plate = 'numberplate.jpg'
                    elif cls != 2:
                        car_type = class_names.get(cls)
                        print(f"Car Type: {car_type}")
                        detected_car_type = car_type

        # Extract text from the image (implement your extract_text_from_image function)
        text = extract_text_from_image(number_plate)
        text = str(text)

        # Define the pattern for number plate extraction using regular expressions
        pattern = r'.*'

        # Search for the pattern in the extracted text
        extracted_number_plates = re.findall(pattern, text)

        # Iterate over the extracted number plates
        for plate in extracted_number_plates:
            extracted_number_plate = str(plate)  # Convert the number plate to a string

            # Check if the number plate is already added
            if extracted_number_plate in added_number_plates:
                continue  # Skip the current iteration if the number plate is already added

            if detected_car_type in added_car_types:
                continue

            # Append the number plate to the added_number_plates list
            added_number_plates.append(extracted_number_plate)

            # Append the car type to the added car type list
            added_car_types.append(detected_car_type)

            # Append the results to the results list
            results_list.append((detected_car_type, extracted_number_plate))

            if mode == "Parking":
                 writer.writerow([detected_car_type, extracted_number_plate.strip("[]"), "1"])  # Convert "1" to string and remove square brackets
            else:
                writer.writerow([detected_car_type, extracted_number_plate.strip("[]"), "0"])  # Convert "0" to string and remove square brackets

# Release the video capture
cap.release()

0: 160x256 1 NumberPlate, 1 Sedan, 382.7ms
Speed: 164.4ms pre-process, 382.7ms inference, 55.4ms postprocess per image at shape (1, 3, 256, 256)


Car Type: Sedan


0: 160x256 1 MidSize-SUV, 1 NumberPlate, 14.7ms
Speed: 1.0ms pre-process, 14.7ms inference, 2.0ms postprocess per image at shape (1, 3, 256, 256)


Car Type: MidSize-SUV


0: 160x256 1 Hatchback, 1 NumberPlate, 13.0ms
Speed: 0.0ms pre-process, 13.0ms inference, 2.0ms postprocess per image at shape (1, 3, 256, 256)


Car Type: Hatchback


0: 160x256 1 NumberPlate, 1 SUV, 16.1ms
Speed: 0.0ms pre-process, 16.1ms inference, 2.4ms postprocess per image at shape (1, 3, 256, 256)


Car Type: SUV
